- !!! COMBINE lr1, lr2 and lr4 || lr1 and lr4 || lr2 and lr4 get high correlation in lr with lr: 0.0001 - 0.01, batch: 10 - 120
- dr--,batch++ && batch+,dr+ && batch+ --> batch_size: high correlation

In [2]:
import pandas as pd 

path = 'reg_data/reg_total.csv'
df = pd.read_csv(path)
df = df[['batch_size', 'learning_rate','dropout', 'val_loss']].dropna()
df

,batch_size,learning_rate,dropout,val_loss
1,83,0.007722,0.160347,0.055907
2,77,0.005789,0.115295,0.057823
3,97,0.008519,0.662069,0.053928
4,79,0.002623,0.194170,0.054212
5,98,0.001041,0.450008,0.052145
...,...,...,...,...
233,132,0.000896,0.681669,0.057568
234,197,0.000765,0.578027,0.056271
235,190,0.000547,0.661905,0.052252
236,216,0.000650,0.624994,0.097364


In [3]:
min_df = df[df['val_loss'] < 0.04]
min_df

,batch_size,learning_rate,dropout,val_loss


In [4]:
import plotly.express as px 
import plotly.graph_objects as go 

fig = go.Figure()
fig.add_traces(
    go.Mesh3d(
        x=df['learning_rate'], y=df['dropout'], z=df['val_loss'], 
        opacity=1,
        intensity=df['batch_size'], colorbar_title='batch_size',
        colorscale='viridis'
        )
    )
fig.add_traces(
    go.Scatter3d(
        x=min_df['learning_rate'], y=min_df['dropout'], z=min_df['val_loss']
    )
)
fig.update_layout(
    scene=dict(
        xaxis_title='learning_rate',
        yaxis_title='dropout',
        zaxis_title='val_loss',
    )
)
fig.show()

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np 


x = df[['batch_size', 'learning_rate', 'dropout']]
y = df['val_loss']
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=1)



In [19]:
degrees = [1,2,3,4,5,6]
mse = [] 
for degree in degrees: 
    poly_reg = PolynomialFeatures(degree=degree)
    poly_x_train = poly_reg.fit_transform(x_train)
    
    regr = LinearRegression()
    regr.fit(poly_x_train, y_train)
    poly_x_val = poly_reg.fit_transform(x_val)
    poly_y_val = regr.predict(poly_x_val)
    mse.append(mean_squared_error(y_val, poly_y_val))



In [20]:
fig = go.Figure()
fig.add_traces(
    go.Scatter(x=degrees, y=mse)
)
fig.update_layout(
    xaxis_title='Degree',
    yaxis_title='Mean square error',
)
fig.show()

In [29]:
degree = 3
poly_reg = PolynomialFeatures(degree=degree)
poly_x_train = poly_reg.fit_transform(x_train)
regr = LinearRegression()
regr.fit(poly_x_train, y_train)
print(x_train.iloc[0])
print(poly_x_train[0])


batch_size       86.000000
learning_rate     0.003315
dropout           0.257375
Name: 30, dtype: float64
[1.00000000e+00 8.60000000e+01 3.31498581e-03 2.57375250e-01
 7.39600000e+03 2.85088780e-01 2.21342715e+01 1.09891309e-05
 8.53195302e-04 6.62420193e-02 6.36056000e+05 2.45176351e+01
 1.90354735e+03 9.45065261e-04 7.33747960e-02 5.69681366e+00
 3.64288132e-08 2.82833032e-06 2.19591354e-04 1.70490563e-02]


In [30]:
coef = regr.coef_
'''
degree = 3: [1,   a,  b,  c, 

            a^2,  ab, ac, b^2, 

            bc, c^2, a^3, a^2.c,

            a^2.b, a.b^2, abc, a.c^2,
            
            b^3  , b^2.c, b.c^2, c^3]

degree = 2: [1,  a,  b,  c,
            a^2, ab, ac, b^2, 
            bc,  c^2]
'''
coef

array([ 0.00000000e+00,  3.52450447e-04, -5.09255005e+00,  2.31728335e-01,
       -1.82809684e-06,  4.65917015e-02, -1.76130692e-03,  1.42062419e+03,
       -1.11984247e+01, -4.42495931e-01,  3.69017850e-09, -9.75351915e-05,
        2.89554602e-06, -3.95710421e+00,  2.45526793e-02,  1.57808990e-03,
       -7.09175573e+04, -4.00947208e+02,  1.54349175e+01,  2.44531548e-01])

In [31]:
poly_x_val = poly_reg.fit_transform(x_val)
poly_y_val = regr.predict(poly_x_val)
mse = mean_squared_error(poly_y_val, y_val)
print(mse)

5.3043425970266384e-05


In [32]:
'''
beta 1->5: the same
beta6 = coef[7]
beta7 = coef[9]
beta8 = coef[5]
beta9 = coef[6]
beta10 = coef[8]
'''
coef_matrix = np.array([[2*coef[4], coef[5], coef[6]],
                        [coef[5], 2*coef[7], coef[8]],
                        [coef[6], coef[8], 2*coef[9]]])
depen_var = np.array([-coef[1], -coef[2], -coef[3]])
np.linalg.solve(coef_matrix, depen_var)

array([4.39692242e+01, 1.67492795e-03, 1.53140936e-01])